# From Notebook to Pipeline

To bring a machine learning model into production we want to automate as much as possible, in a reproducible way.
Therefore, production grade code should be in a proper versioned repository like git and not in a notebook.

In this tutorial, we are going to turn a prototype from a notebook into a pipeline on azure machine learning.

The goal is:
* turn an existing model into a service from which other services can use to make predictions
* extend model training so that it can run independently of your machine in the cloud and training can be automated further

This is the first step on the way to create a production grade ML application in the cloud.

Not part of this tutorial is the integration of the azure machine learning pipelines into a CI/CD pipeline.
If you are interested what a CI/CD pipeline is, you can ask me and I will show you an example.



## Overview
Azure Machine Learning python sdk gives us a relatively simple interface to run model training jobs and create deployments for inference.

You can think of it as a job scheduling tool.
Additionally to the code that trains a model and serves it, we will write job scheduling code. We will write code that creates and runs training jobs, creates and deploys  REST endpoints and creates docker images.

**We assume that you already have an azure machine learning workspace.**


## Install Environment

Before starting you should install all required dependencies locally. We will do so in a virtual environment:
```bash
python -m venv azml-tutorial-venv
```

Activate the virtual environment.
On unix like systems:
```bash
source azml-tutorial-env/bin/activate
```
On Windows:
```ps1
.\azml-tutorial-env\Scripts\activate
```


Then register this venv with jupyter so we can use it within the notebook.
Do so by running this code inside the activated venv:
```bash
python -m ipykernel install --user --name=azml-tutorial
```

Restart the notebook with the kernel `azml-tutorial` before you continue.

## Connect to Azure Machine Learning

We will create the client that handles all requests to azure machine learning.
### Authenticate
There are different methods to authenticate. Choose the one that works for you:



In [ ]:
import os
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential

token_url = "https://management.azure.com/.default"
tenant_id = "" # set this to the id of your tenant

Depending on your circumstances you can use different ways of authenticating to Azure Machine Learning. Here are a few:

```python
credential = DefaultAzureCredential();
credential.get_token(token_url, tenant_id=tenant_id);

credential = AzureCliCredential();
credential.get_token(token_url);

# via browser
credential = InteractiveBrowserCredential(tenant_id=tenant_id);
credential.get_token(token_url);

print("created the following credential: ", credential)
```

When running the notebook on you local machine it might be easier to authenticate via the browser:

In [ ]:
credential = InteractiveBrowserCredential(tenant_id=tenant_id);
credential.get_token(token_url);

When you are running the notebook from azure machine learning, you can authenticate first in the terminal via `az login` and then continue with the `AzureCliCredential`:

In [ ]:
credential = AzureCliCredential();
credential.get_token(token_url);

### Create a client to connect to the workspace
This is the central object to interact with Azure Machine Learning programatically.

Please fill it in so that it matches your worspace:

In [ ]:
from azure.ai.ml import MLClient

ml_client = MLClient(
    credential=credential,
    subscription_id='',
    resource_group_name='',
    workspace_name='',
)

In [ ]:
list(ml_client.data.list())

**You don't need to go through the Environments section for the tutorial to work.**
You can skip it an look at it later.

## Run Jobs
We will now try to give you a basic understanding of how to run jobs and how they work.
A job will run the script you specify (or any command available from the environment). 
When you schedule a job it will get a vm and start it with the specified environment, upload the script to azure and run the command you have specified.

A job can have inputs and outputs. the azure storage assets referenced in the inputs and outputs will be mounted into the filesystem of the VM of the job. Therefore, your script can read from the `Input` paths write to an `Output` paths as if it were ordinary files and directories. 

Here a simple example of how you can run the script `train_jonb.py` on azure machine learning:

In [ ]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes


input_data = {"dataset": Input(type=AssetTypes.URI_FOLDER, path="azureml:name-of-dataset:1")}
output_data = {'model': Output(type=AssetTypes.CUSTOM_MODEL)}

train_job = command(
    code="./",
    command="pip install -r requirements.txt && python train_job.py --input-asset ${{inputs.dataset}} --output-asset ${{outputs.model}}",
    environment="AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu:3",
    compute="cheap-standard-d1-v2",
    inputs=input_data,
    outputs=output_data
)

From the code above, we can see, that
* we need a dataset called `name-of-dataset` needs to exist.
 * Create in the azure machine learning sutudio an example dataset or take the name of an existing one.
* we need a compute cluster
 * change the line so it matches your compute cluster or create one via the studio UI


Now we create the script `train_job.py`. Play around with it and make some changes and run the job to get used to how it works. 

Running the following cell will overwrite `train_job.py` so make sure you do not change it locally!

In [ ]:
%%writefile train_job.py
import argparse
import cloudpickle
import os

def parse_args():
    """Read command line arguments"""
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-asset", type=str)
    parser.add_argument("--output-asset", type=str)
    return parser.parse_args()

args = parse_args()
print("input_asset:", args.input_asset)
print("output_asset:", args.output_asset)

print("input dataset files:", os.listdir(args.input_asset))

class Model:
    def __init__(self, dummy):
        self.dummy = dummy

model = Model("dummy model.")
        
# save you model within the args.output_asset path

##os.makedirs(args.output_asset+"model/", exist_ok=True)
with open(args.output_asset+"/model.pickle", "wb") as fh:
    cloudpickle.dump(model, fh) # with joblib you can save almost any python object to disk
    



You can run the script locally:

In [ ]:
%run train_job.py --input-asset ./ --output-asset ./

Now we run the job in the cloud:

In [ ]:
job = ml_client.jobs.create_or_update(train_job)
job

The link leads you to the currently running job in Azure ML Studio.
Check it out and watch the job starting.
You can look at the logs unter `Outputs + Logs` one of the shows the logs generated by our script.

### Tasks:
* Make yourself familiar with the job interface you can see the code, inputs and outputs
* Check the logs of the job, you should find the output of the print statements in one of the logfiles


## Task: Train a real Model
You can now use our job running capability to create a model.
Create a script similar to `train_job.py` that trains your model and save it into the directory you specified in the `outputs`.
* take the code from the notebook that defines the model and trains it and put it into `your_train_job.py` or give it some other name
* for now, train the model with the minimal amount of data necessary
* define any parameters of this script that you may want to change via commandline arguments
* before running the job on azure, run `your_train_job.py` from commandline with the necessary arguments locally
 * weed out errors
* set the arguments in `command()` and define the input and output dataset accordingly
* save the model into the `outputs` if joblib fails, 
* run the job and check if it runs as intended
* check that the model output is created by looking at the outputs of the jobs
* register the model you created manually via the studio ui (this could also be done via the sdk)
 * give the model a proper name


## Inference
The next step is to run our model as a service and make predictions.

Azure Machine learning allows us to run a model as a REST service by creating a simple wrapper of the model.

**Make sure that you registered your model** You can do this via the studio UI. Select the job and there `+Register Model`.

We will do it in the following.
Read the comments to understand how it works:

In [ ]:
%%writefile inference.py
import os
import cloudpickle # a bit better than joblib
import numpy as np
import json

def init():
    global model # this variable can be accessed outside this function

    model_dir = os.getenv("AZUREML_MODEL_DIR") #when the script runs on azure, it will have this environment variable, it contains the path to the model in the VM file system
    if model_dir == None:
        model_dir = "" # define here the path to the model on your machine if you want to test the script locally
    model_path = os.path.join(model_dir, "model/model.pickle") # specify here the exact path to the model file inside the storage
    with open(model_path, "rb") as fh:
        model = cloudpickle.load(fh) # load the model, you can also use joblib

        
def run(input_data):
    data = json.loads(input_data) # interpret the input to the rest service as json
    # parse the json according to your needs and feed the input to the model
    # for example you could encode the image data as base64 encoded string:
    # # read the string:
    # image = base64.decodestring(data['image'])
    # This might not be the architecturally best way of using the service in a real live system.
    # I'm happy to discuss how to handle the image data more properly :-)
    return {"answer": 42, "input": data} # here we send the input back and add THE answer


if __name__ == "__main__":
    #the code here you could use to test the functions above on your machine
    init()
    print(run('''{"question": "Answer to the Ultimate Question of Life, The Universe, and Everything"}'''))


Try out the script by running it locally:

In [ ]:
%run inference.py

When running the script on azure machine learning it will run `init()` on startup and `run()` every time the endpoint is called.

### Create the inference endpoint and deploy the script
We will create a endpoint that is managed by azure machine learning. After the endpoint was created there will be a representation of it in the sudio UI where you can try the model out and generate code to call the endpoint from the internet as well as manage the API key.

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration

# If you want to setup the endpoint locally you need to set local=True
# for this to work you need docker installed and running on your machine
local=False 
online_endpoint_name = "endpoint-service-test"

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Test Endpoint for MLOps Tutorial",
    auth_mode="key"
)

endpoint_poller = ml_client.begin_create_or_update(endpoint, local=local)

if local: # if you want to upload a model from your machine set local to true
    model = Model(path="model/model.pickle")
else:
    model=ml_client.models.get(name="dummy-model", version="1") # reference the registered model by its name and version

code_config = CodeConfiguration(
    code="./", scoring_script="inference.py" # change if you want to run another script
)

# here you could also take a custom environment for inference that already has the exact dependencies you need
environment = "AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu:3"

# you could have multiple deployments per endpoint
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=environment,
    code_configuration=code_config,
    instance_type="Standard_F2s_v2", #change here if you need a gpu for inference
    instance_count=1,
)

if not local:
    endpoint_poller.result() # wait for the endpoint to be created

deployment_poller = ml_client.begin_create_or_update(blue_deployment, local=local)
deployment_poller.result() #wait for the deployment to be finished

# blue deployment gets 100% traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint, local=runner_config.local)


Now go to the endpoint in the UI and send a test request according to the input format defined above.

## Task: Create your own Model Endpoint
* cpoy `inference.py` to `your_inference.py` and adjust it according to your needs
* load your model
* adjust `run()` such that it takes the input data the way you need it
  * for example read it from a base64 encoded image object

## Environments
If you want to create your own environments you can continue here.

The role of environments is to manage and define the contents of the virtual machine where your code will run.

It should contain all dependencies of the job at hand.

### Types
Two types of environments where your jobs will run:
- existing environment
- custom environment
  - conda environment
  - docker environment

Use a custom environment when:
- you have dependencies that take a lot of time to install
- you need non-python packages or system libraries (via docker)


With a custom environment the dependencies are installed once into the docker container. Custom environments can be based on an existing environment.

### Existing Environment
When you are using an existing environment, you can also have your own dependencies, but they will be installed every time you are running a job.
Example:

In [ ]:
from azure.ai.ml import command

job_with_existing_env_with_custom_deps = command(
    code="./",
    command="pip install -r requirements.txt && python script.py",
    environment="AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu:35"
)



This will install the dependencies listed in requeirements every time you run the job.
You can look up existing environments in Azure Machine Learning Studio under "Environments".

You could run this job with: 
```python
ml_client.environments.create_or_update(existing_env_with_custom_deps)
```


### Custom Environments
Create an environment that contains the dependencies you need.
In general this will build a docker container with everything that your app will need.

There is a simple and a advanced way of creating an environment:
- by supplying conda dependnecies
- by creating your own docker file with whatever dependencies you will need

To create a custom environment from conda dependencies you need to reference your "environment.yml" file containing the conda packages you need.

```python
from azure.ai.ml.entities import Environment
conda_env = Environment(
    conda_file="environment.yml",
    name="tutorial-ubuntu2004-py310-cpu",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    description="conda environment"
)
```

To actually build the environment you need to send it to the workspace:
```pathon
ml_client.environments.create_or_update(conda_env)
```

The most flexibility you have with the Docker Environment. Here you can specify the path to your `Dockerfile`.

```python
from azure.ai.ml.entities import BuildContext
docker_env = Environment(
    build=BuildContext(path="path/to/directory/where/dockerfile/is/located", dockerfile_path="Dockerfile"),
    name="tutorial-advanced-ubuntu2004-py310-cpu",
    description="advanced environment"
)
```

With this you can create your custom docker image.

### Create Project Environment

We will now create a custom environment for our project. In the notebook we are using pip to install packages. It would be easier to create a conda environment, but this would need testing first.

Therefore, we are going to use pip and install the packages into a custom docker container.


In [ ]:
from azure.ai.ml.entities import Environment, BuildContext
docker_env = Environment(
    build=BuildContext(path="generated/docker", dockerfile_path="Dockerfile"),
    name="tutorial-advanced-ubuntu2004-py310-cpu",
    description="advanced environment"
)

Let us now create a corresponding `Dockerfile`:

In [ ]:
%%writefile Dockerfile
# this file is mostly a copy of the Dockerfile of the image "AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu". You can find it under "Environments" on Azure Machine Learning Sutdio
FROM mcr.microsoft.com/azureml/aifx/stable-ubuntu2004-cu116-py39-torch1121:biweekly.202211.1

# Install pip dependencies
RUN pip install 'ipykernel~=6.0' \
                'azureml-core==1.47.0' \
				'azureml-dataset-runtime==1.47.0' \
                'azureml-defaults==1.47.0' \
				'azure-ml==0.0.1' \
				'azure-ml-component==0.9.15.post2' \
                'azureml-mlflow==1.47.0' \
		'azureml-contrib-services==1.47.0' \
		        'azureml-contrib-services==1.47.0' \
                'torch-tb-profiler~=0.4.0' \
				'py-spy==0.3.12' \
		        'debugpy~=1.6.3'

RUN pip install \
        azure-ai-ml==0.1.0b5 \
        MarkupSafe==2.1.1 \
	    regex \
	    pybind11

# Inference requirements
COPY --from=mcr.microsoft.com/azureml/o16n-base/python-assets:20220607.v1 /artifacts /var/
RUN /var/requirements/install_system_requirements.sh && \
    cp /var/configuration/rsyslog.conf /etc/rsyslog.conf && \
    cp /var/configuration/nginx.conf /etc/nginx/sites-available/app && \
    ln -sf /etc/nginx/sites-available/app /etc/nginx/sites-enabled/app && \
    rm -f /etc/nginx/sites-enabled/default
ENV SVDIR=/var/runit
ENV WORKER_TIMEOUT=400
EXPOSE 5001 8883 8888

# support Deepspeed launcher requirement of passwordless ssh login

RUN apt-get update
RUN apt-get install -y openssh-server openssh-client
RUN mkdir -p /root/.ssh
RUN mkdir /var/run/sshd
RUN ssh-keygen -t rsa -f /root/.ssh/id_rsa
RUN sed -i 's/PermitRootLogin without-password/PermitRootLogin yes/' /etc/ssh/sshd_config
RUN sed 's@session\\s*required\\s*pam_loginuid.so@session optional pam_loginuid.so@g' -i /etc/pam.d/sshd
RUN chmod 700 /root/.ssh/
RUN touch /root/.ssh/config;echo -e "Port 1043\n StrictHostKeyChecking no\n  UserKnownHostsFile=/dev/null" > /root/.ssh/config
RUN echo "Port 1043" >> /etc/ssh/sshd_config
RUN chmod 600 /root/.ssh/config
RUN touch /root/.ssh/authorized_keys && chmod 600 /root/.ssh/authorized_keys
RUN cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys
EXPOSE 1043 1043
CMD ["/usr/sbin/sshd", "-D"]

#here our command to install our requirements: